In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from baseline.raif_hack.metrics_to_test import deviation_metric
from tqdm import tqdm
import re

In [2]:
train_data = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

C:\Users\ann\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
osm_cols = [q for q in train_data.columns if ('osm' in q) and ('name' not in q)]

In [4]:
osm_cols = [q for q in osm_cols if (len(re.findall('\d', q)) == 0 ) or ('005' in q)]
osm_cols_to_drop = [q for q in train_data.columns if ('osm' in q) and (q not in osm_cols)]

In [5]:
from catboost import CatBoostRegressor, Pool

In [6]:
def change_floor(x):
    try:
        return int(float(x))
    except:
        nums = re.findall('((?<!\d)-?\d+\.?\d?)', str(x))
        if len(nums) > 0:
            return np.mean([float(d) for d in nums])
        if 'подва' in x:
            return -1
        if 'манса' in x:
            return 4
        if 'цок' in x:
            return -1
        return np.nan

In [8]:
#transform train_data

train_data['floor'] = train_data.floor.map(lambda x: change_floor(str(x)))
train_data.loc[train_data['floor'] > 80, 'floor'] = 80
train_data['date'] = pd.to_datetime(train_data.date)
train_data['day'] = (train_data.date - pd.Timestamp(2010, 1, 1)).map(lambda x: x.days)

In [10]:
#transform test data
test['floor'] = test.floor.map(lambda x: change_floor(str(x)))
test.loc[test['floor'] > 80, 'floor'] = 80
test['day'] = (pd.to_datetime(test.date) - pd.Timestamp(2010, 1, 1)).map(lambda x: x.days)

In [12]:
target_col = 'per_square_meter_price'
cols_to_drop = ['id', 'date', 'price_type'] + [target_col] + osm_cols_to_drop + osm_cols
cols = train_data.drop(columns=cols_to_drop).columns
cat_feat_idx = list(train_data.drop(columns=cols_to_drop).dtypes.astype(str)
                    .to_frame('tp').reset_index()
                    .sort_values(by=['tp', 'index'])
                    .query("tp != 'int64' & tp != 'float64'").index)

In [44]:
fill_val = train_data.drop(columns='date').median()
train = train_data.fillna({cols[q]: 'missing' for q in cat_feat_idx}).fillna(dict(fill_val))
test = test.fillna({cols[q]: 'missing' for q in cat_feat_idx}).fillna(dict(fill_val))

In [49]:
valid_date = pd.Timestamp(2020, 8, 13)
valid = train.query("date > @valid_date & price_type == 1")

In [50]:
train.loc[:, 'target_log'] = np.log(train[target_col])
valid.loc[:, 'target_log'] = np.log(valid[target_col])

C:\Users\ann\Anaconda3\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [56]:
train['closest_dist_min'] = train[['osm_train_stop_closest_dist', 'osm_transport_stop_closest_dist']].min(axis=1)
train['closest_stop_sum'] = train['osm_train_stop_points_in_0.005'] * train['osm_transport_stop_points_in_0.005']
train['some_sum'] = train[['osm_historic_points_in_0.005',
                     'osm_culture_points_in_0.005',
                     'osm_hotels_points_in_0.005']].sum(axis=1)
train['total_square'] = np.log(train['total_square'])
train['realty_type'] = train['realty_type'].astype(str)

In [51]:
valid['closest_dist_min'] = valid[['osm_train_stop_closest_dist', 'osm_transport_stop_closest_dist']].min(axis=1)
valid['closest_stop_sum'] = valid['osm_train_stop_points_in_0.005'] * valid['osm_transport_stop_points_in_0.005']
valid['some_sum'] = valid[['osm_historic_points_in_0.005',
                     'osm_culture_points_in_0.005',
                     'osm_hotels_points_in_0.005']].sum(axis=1)
valid['total_square'] = np.log(valid['total_square'])
valid['realty_type'] = valid['realty_type'].astype(str)

C:\Users\ann\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\ann\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ann\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [31]:
test['closest_dist_min'] = test[['osm_train_stop_closest_dist', 'osm_transport_stop_closest_dist']].min(axis=1)
test['closest_stop_sum'] = test['osm_train_stop_points_in_0.005'] * test['osm_transport_stop_points_in_0.005']
test['some_sum'] = test[['osm_historic_points_in_0.005',
                     'osm_culture_points_in_0.005',
                     'osm_hotels_points_in_0.005']].sum(axis=1)
test['total_square'] = np.log(test['total_square'])
test['realty_type'] = test['realty_type'].astype(str)

In [34]:
cols_up = np.hstack([cols, ['closest_dist_min', 'closest_stop_sum', 'some_sum']])

In [68]:
target_col = 'per_square_meter_price'
train_idx = (train.price_type == 1) & (train.date <= valid_date)
train_pool = Pool(train.loc[train_idx, cols_up],
                  label=np.log(train.loc[train_idx, target_col]),
                  cat_features=cat_feat_idx)
valid_pool = Pool(valid[cols_up], valid['target_log'], cat_features=cat_feat_idx)
test_pool = Pool(test[cols_up], cat_features=cat_feat_idx)

In [70]:
model = CatBoostRegressor(iterations=1800, learning_rate=.012, depth=6, verbose=True, random_state=42, loss_function='MAPE')
model.fit(train_pool, plot=True, eval_set=valid_pool)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0371001	test: 0.0383653	best: 0.0383653 (0)	total: 86.3ms	remaining: 2m 35s
1:	learn: 0.0369327	test: 0.0381889	best: 0.0381889 (1)	total: 173ms	remaining: 2m 35s
2:	learn: 0.0367651	test: 0.0380216	best: 0.0380216 (2)	total: 250ms	remaining: 2m 29s
3:	learn: 0.0366227	test: 0.0378921	best: 0.0378921 (3)	total: 335ms	remaining: 2m 30s
4:	learn: 0.0364563	test: 0.0377236	best: 0.0377236 (4)	total: 416ms	remaining: 2m 29s
5:	learn: 0.0362964	test: 0.0375683	best: 0.0375683 (5)	total: 492ms	remaining: 2m 27s
6:	learn: 0.0361435	test: 0.0373951	best: 0.0373951 (6)	total: 575ms	remaining: 2m 27s
7:	learn: 0.0359800	test: 0.0372399	best: 0.0372399 (7)	total: 660ms	remaining: 2m 27s
8:	learn: 0.0358655	test: 0.0371284	best: 0.0371284 (8)	total: 699ms	remaining: 2m 19s
9:	learn: 0.0357121	test: 0.0369561	best: 0.0369561 (9)	total: 786ms	remaining: 2m 20s
10:	learn: 0.0355658	test: 0.0367996	best: 0.0367996 (10)	total: 869ms	remaining: 2m 21s
11:	learn: 0.0354231	test: 0.0366561	bes

93:	learn: 0.0281102	test: 0.0292895	best: 0.0292895 (93)	total: 7.93s	remaining: 2m 24s
94:	learn: 0.0280558	test: 0.0292428	best: 0.0292428 (94)	total: 8.03s	remaining: 2m 24s
95:	learn: 0.0279988	test: 0.0291939	best: 0.0291939 (95)	total: 8.12s	remaining: 2m 24s
96:	learn: 0.0279566	test: 0.0291520	best: 0.0291520 (96)	total: 8.2s	remaining: 2m 24s
97:	learn: 0.0279145	test: 0.0291128	best: 0.0291128 (97)	total: 8.29s	remaining: 2m 23s
98:	learn: 0.0278762	test: 0.0290811	best: 0.0290811 (98)	total: 8.35s	remaining: 2m 23s
99:	learn: 0.0278307	test: 0.0290263	best: 0.0290263 (99)	total: 8.41s	remaining: 2m 23s
100:	learn: 0.0277841	test: 0.0289818	best: 0.0289818 (100)	total: 8.48s	remaining: 2m 22s
101:	learn: 0.0277422	test: 0.0289390	best: 0.0289390 (101)	total: 8.55s	remaining: 2m 22s
102:	learn: 0.0276909	test: 0.0288963	best: 0.0288963 (102)	total: 8.61s	remaining: 2m 21s
103:	learn: 0.0276310	test: 0.0288339	best: 0.0288339 (103)	total: 8.68s	remaining: 2m 21s
104:	learn: 0.

186:	learn: 0.0250614	test: 0.0266833	best: 0.0266833 (186)	total: 14.5s	remaining: 2m 4s
187:	learn: 0.0250393	test: 0.0266694	best: 0.0266694 (187)	total: 14.6s	remaining: 2m 4s
188:	learn: 0.0250182	test: 0.0266605	best: 0.0266605 (188)	total: 14.6s	remaining: 2m 4s
189:	learn: 0.0249994	test: 0.0266432	best: 0.0266432 (189)	total: 14.7s	remaining: 2m 4s
190:	learn: 0.0249775	test: 0.0266247	best: 0.0266247 (190)	total: 14.8s	remaining: 2m 4s
191:	learn: 0.0249571	test: 0.0266118	best: 0.0266118 (191)	total: 14.8s	remaining: 2m 4s
192:	learn: 0.0249351	test: 0.0266038	best: 0.0266038 (192)	total: 14.9s	remaining: 2m 4s
193:	learn: 0.0249149	test: 0.0265857	best: 0.0265857 (193)	total: 15s	remaining: 2m 4s
194:	learn: 0.0248953	test: 0.0265680	best: 0.0265680 (194)	total: 15.1s	remaining: 2m 3s
195:	learn: 0.0248767	test: 0.0265519	best: 0.0265519 (195)	total: 15.1s	remaining: 2m 3s
196:	learn: 0.0248549	test: 0.0265386	best: 0.0265386 (196)	total: 15.2s	remaining: 2m 3s
197:	learn: 

...

1374:	learn: 0.0184342	test: 0.0240772	best: 0.0240772 (1374)	total: 1m 52s	remaining: 34.9s
1375:	learn: 0.0184308	test: 0.0240779	best: 0.0240772 (1374)	total: 1m 52s	remaining: 34.8s
1376:	learn: 0.0184283	test: 0.0240756	best: 0.0240756 (1376)	total: 1m 53s	remaining: 34.7s
1377:	learn: 0.0184257	test: 0.0240752	best: 0.0240752 (1377)	total: 1m 53s	remaining: 34.6s
1378:	learn: 0.0184226	test: 0.0240760	best: 0.0240752 (1377)	total: 1m 53s	remaining: 34.5s
1379:	learn: 0.0184201	test: 0.0240751	best: 0.0240751 (1379)	total: 1m 53s	remaining: 34.5s
1380:	learn: 0.0184183	test: 0.0240758	best: 0.0240751 (1379)	total: 1m 53s	remaining: 34.4s
1381:	learn: 0.0184166	test: 0.0240737	best: 0.0240737 (1381)	total: 1m 53s	remaining: 34.3s
1382:	learn: 0.0184125	test: 0.0240725	best: 0.0240725 (1382)	total: 1m 53s	remaining: 34.2s
1383:	learn: 0.0184102	test: 0.0240705	best: 0.0240705 (1383)	total: 1m 53s	remaining: 34.1s
1384:	learn: 0.0184058	test: 0.0240682	best: 0.0240682 (1384)	total: 1

1465:	learn: 0.0182110	test: 0.0240522	best: 0.0240442 (1457)	total: 2m	remaining: 27.5s
1466:	learn: 0.0182090	test: 0.0240531	best: 0.0240442 (1457)	total: 2m	remaining: 27.4s
1467:	learn: 0.0182045	test: 0.0240715	best: 0.0240442 (1457)	total: 2m	remaining: 27.3s
1468:	learn: 0.0182019	test: 0.0240726	best: 0.0240442 (1457)	total: 2m	remaining: 27.2s
1469:	learn: 0.0181989	test: 0.0240680	best: 0.0240442 (1457)	total: 2m	remaining: 27.1s
1470:	learn: 0.0181978	test: 0.0240676	best: 0.0240442 (1457)	total: 2m	remaining: 27.1s
1471:	learn: 0.0181939	test: 0.0240663	best: 0.0240442 (1457)	total: 2m 1s	remaining: 27s
1472:	learn: 0.0181931	test: 0.0240664	best: 0.0240442 (1457)	total: 2m 1s	remaining: 26.9s
1473:	learn: 0.0181906	test: 0.0240573	best: 0.0240442 (1457)	total: 2m 1s	remaining: 26.8s
1474:	learn: 0.0181887	test: 0.0240568	best: 0.0240442 (1457)	total: 2m 1s	remaining: 26.8s
1475:	learn: 0.0181868	test: 0.0240561	best: 0.0240442 (1457)	total: 2m 1s	remaining: 26.7s
1476:	le

1557:	learn: 0.0179916	test: 0.0240032	best: 0.0239997 (1554)	total: 2m 8s	remaining: 20s
1558:	learn: 0.0179900	test: 0.0240048	best: 0.0239997 (1554)	total: 2m 8s	remaining: 19.9s
1559:	learn: 0.0179866	test: 0.0240014	best: 0.0239997 (1554)	total: 2m 8s	remaining: 19.8s
1560:	learn: 0.0179830	test: 0.0240005	best: 0.0239997 (1554)	total: 2m 8s	remaining: 19.7s
1561:	learn: 0.0179812	test: 0.0239999	best: 0.0239997 (1554)	total: 2m 8s	remaining: 19.6s
1562:	learn: 0.0179790	test: 0.0240031	best: 0.0239997 (1554)	total: 2m 8s	remaining: 19.5s
1563:	learn: 0.0179755	test: 0.0239989	best: 0.0239989 (1563)	total: 2m 8s	remaining: 19.5s
1564:	learn: 0.0179736	test: 0.0239954	best: 0.0239954 (1564)	total: 2m 9s	remaining: 19.4s
1565:	learn: 0.0179729	test: 0.0239966	best: 0.0239954 (1564)	total: 2m 9s	remaining: 19.3s
1566:	learn: 0.0179695	test: 0.0239955	best: 0.0239954 (1564)	total: 2m 9s	remaining: 19.3s
1567:	learn: 0.0179688	test: 0.0239953	best: 0.0239953 (1567)	total: 2m 9s	remaini

1647:	learn: 0.0177930	test: 0.0239801	best: 0.0239707 (1609)	total: 2m 17s	remaining: 12.6s
1648:	learn: 0.0177918	test: 0.0239796	best: 0.0239707 (1609)	total: 2m 17s	remaining: 12.6s
1649:	learn: 0.0177899	test: 0.0239778	best: 0.0239707 (1609)	total: 2m 17s	remaining: 12.5s
1650:	learn: 0.0177887	test: 0.0239785	best: 0.0239707 (1609)	total: 2m 17s	remaining: 12.4s
1651:	learn: 0.0177855	test: 0.0239786	best: 0.0239707 (1609)	total: 2m 17s	remaining: 12.3s
1652:	learn: 0.0177830	test: 0.0239777	best: 0.0239707 (1609)	total: 2m 17s	remaining: 12.2s
1653:	learn: 0.0177810	test: 0.0239774	best: 0.0239707 (1609)	total: 2m 17s	remaining: 12.2s
1654:	learn: 0.0177782	test: 0.0239796	best: 0.0239707 (1609)	total: 2m 17s	remaining: 12.1s
1655:	learn: 0.0177764	test: 0.0239793	best: 0.0239707 (1609)	total: 2m 17s	remaining: 12s
1656:	learn: 0.0177749	test: 0.0239797	best: 0.0239707 (1609)	total: 2m 17s	remaining: 11.9s
1657:	learn: 0.0177733	test: 0.0239790	best: 0.0239707 (1609)	total: 2m 

1737:	learn: 0.0176053	test: 0.0239592	best: 0.0239560 (1716)	total: 2m 25s	remaining: 5.2s
1738:	learn: 0.0176020	test: 0.0239588	best: 0.0239560 (1716)	total: 2m 25s	remaining: 5.12s
1739:	learn: 0.0176011	test: 0.0239592	best: 0.0239560 (1716)	total: 2m 25s	remaining: 5.03s
1740:	learn: 0.0175987	test: 0.0239598	best: 0.0239560 (1716)	total: 2m 26s	remaining: 4.95s
1741:	learn: 0.0175968	test: 0.0239595	best: 0.0239560 (1716)	total: 2m 26s	remaining: 4.86s
1742:	learn: 0.0175953	test: 0.0239595	best: 0.0239560 (1716)	total: 2m 26s	remaining: 4.78s
1743:	learn: 0.0175936	test: 0.0239597	best: 0.0239560 (1716)	total: 2m 26s	remaining: 4.7s
1744:	learn: 0.0175919	test: 0.0239591	best: 0.0239560 (1716)	total: 2m 26s	remaining: 4.61s
1745:	learn: 0.0175903	test: 0.0239639	best: 0.0239560 (1716)	total: 2m 26s	remaining: 4.53s
1746:	learn: 0.0175881	test: 0.0239651	best: 0.0239560 (1716)	total: 2m 26s	remaining: 4.44s
1747:	learn: 0.0175850	test: 0.0239639	best: 0.0239560 (1716)	total: 2m 

In [71]:
train_p = np.exp(model.predict(train.loc[train_idx, cols_up]))
train.loc[train_idx, 'delta'] = ((train.loc[train_idx, target_col] - train_p)/ train_p)
corr_koef_city = dict(train.query("price_type == 1").groupby(['city']).delta.median())
corr_koef_region = dict(train.query("price_type == 1").groupby(['region']).delta.median())
corr_koef = train.query("price_type == 1").groupby(['city']).delta.median().median()

In [300]:
pd.DataFrame({'imp':model.feature_importances_,
              'feat': model.feature_names_})\
    .sort_values(by='imp', ascending=False)\
    .assign(cs=lambda x: x.imp.cumsum())
#     .query("cs < 80")\
#     .set_index('feat')\
#     .index

,imp,feat,cs
0,14.583976,city,14.583976
33,14.277132,total_square,28.861108
32,11.996213,region,40.857321
6,8.983460,osm_catering_points_in_0.005,49.840781
8,5.828190,osm_city_nearest_population,55.668971
4,4.320105,osm_amenity_points_in_0.005,59.989077
19,3.589707,osm_subway_closest_dist,63.578783
1,3.496198,floor,67.074982
15,3.041495,osm_hotels_points_in_0.005,70.116477
35,2.809316,realty_type,72.925793


In [72]:
valid_p = np.exp(model.predict(valid_pool))
valid['corr_koef'] = (valid[['city', 'region']]
                          .apply(lambda x: corr_koef_city.get(x[0], corr_koef_region.get(x[1], corr_koef)),
                                 axis=1)
                      + 1)
valid_p = (valid_p * valid['corr_koef'])

C:\Users\ann\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [74]:
# deviation_metric(valid[target_col].values, valid_p), deviation_metric(train_data.loc[idx_0, target_col].values, train_p.va)
deviation_metric(valid[target_col].values, valid_p.values), deviation_metric(train.loc[train_idx, target_col].values, train_p)

(1.4717096744178317, 0.990539967121378)

In [75]:
test_p = np.exp(model.predict(test_pool))
test['corr_koef'] = (test[['city', 'region']]
                          .apply(lambda x: corr_koef_city.get(x[0], corr_koef_region.get(x[1], corr_koef)),
                                 axis=1)
                      + 1)
test_p = (test_p * test['corr_koef']) * .92

In [76]:
test[target_col] = test_p

In [77]:
test[['id', target_col]].to_csv("./output/try_10.csv", sep=',', header=True, index=False)

In [68]:
to_drop_2 = list(set(cols) - set(cols2)) + ['date', 'delta', 'id'] + [target_col]
cols2 = train.drop(columns=to_drop_2, errors='ignore').columns

In [69]:
cat_feat_idx_2 = list(train.drop(columns=to_drop_2, errors='ignore').dtypes.astype(str)
                    .to_frame('tp').reset_index()
                    .sort_values(by=['tp', 'index'])
                    .query("tp != 'int64' & tp != 'float64'").index)

In [ ]:

valid_pool = Pool(valid[cols], valid[target_col], cat_features=cat_feat_idx)

In [92]:
train_pool_2 = Pool(train[cols2], train[target_col], cat_features=cat_feat_idx_2)
valid_pool_2 = Pool(valid[cols2], valid[target_col], cat_features=cat_feat_idx_2)

In [99]:
model2 = CatBoostRegressor(iterations=2000, learning_rate=.02, depth=4,
                          verbose=True, random_state=42,
                           eval_metric='MAPE')
model2.fit(train_pool_2, plot=True,
           eval_set=valid_pool_2)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 4.6493958	test: 1.5439773	best: 1.5439773 (0)	total: 579ms	remaining: 19m 16s
1:	learn: 4.6013779	test: 1.5185261	best: 1.5185261 (1)	total: 1.07s	remaining: 17m 49s
2:	learn: 4.5555786	test: 1.4949303	best: 1.4949303 (2)	total: 1.36s	remaining: 15m 7s
3:	learn: 4.5105508	test: 1.4720640	best: 1.4720640 (3)	total: 1.78s	remaining: 14m 50s
4:	learn: 4.4661208	test: 1.4491364	best: 1.4491364 (4)	total: 1.98s	remaining: 13m 10s
5:	learn: 4.4233447	test: 1.4272031	best: 1.4272031 (5)	total: 2.21s	remaining: 12m 14s
6:	learn: 4.3791288	test: 1.4037995	best: 1.4037995 (6)	total: 2.56s	remaining: 12m 10s
7:	learn: 4.3385610	test: 1.3844327	best: 1.3844327 (7)	total: 2.91s	remaining: 12m 3s
8:	learn: 4.2995787	test: 1.3652412	best: 1.3652412 (8)	total: 3.19s	remaining: 11m 45s
9:	learn: 4.2599884	test: 1.3452871	best: 1.3452871 (9)	total: 3.51s	remaining: 11m 38s
10:	learn: 4.2239027	test: 1.3278893	best: 1.3278893 (10)	total: 3.73s	remaining: 11m 13s
11:	learn: 4.1899091	test: 1.310

93:	learn: 2.6171155	test: 0.5809583	best: 0.5809583 (93)	total: 18.5s	remaining: 6m 15s
94:	learn: 2.6070576	test: 0.5774642	best: 0.5774642 (94)	total: 18.7s	remaining: 6m 15s
95:	learn: 2.6006864	test: 0.5746015	best: 0.5746015 (95)	total: 18.9s	remaining: 6m 14s
96:	learn: 2.5910906	test: 0.5702285	best: 0.5702285 (96)	total: 19s	remaining: 6m 13s
97:	learn: 2.5843637	test: 0.5676720	best: 0.5676720 (97)	total: 19.2s	remaining: 6m 13s
98:	learn: 2.5765910	test: 0.5642278	best: 0.5642278 (98)	total: 19.4s	remaining: 6m 12s
99:	learn: 2.5671619	test: 0.5619496	best: 0.5619496 (99)	total: 19.6s	remaining: 6m 12s
100:	learn: 2.5558515	test: 0.5566126	best: 0.5566126 (100)	total: 19.8s	remaining: 6m 11s
101:	learn: 2.5487328	test: 0.5538501	best: 0.5538501 (101)	total: 20s	remaining: 6m 12s
102:	learn: 2.5409751	test: 0.5510692	best: 0.5510692 (102)	total: 20.2s	remaining: 6m 11s
103:	learn: 2.5337629	test: 0.5486592	best: 0.5486592 (103)	total: 20.4s	remaining: 6m 11s
104:	learn: 2.524

185:	learn: 2.0386663	test: 0.4031202	best: 0.4031202 (185)	total: 34.5s	remaining: 5m 36s
186:	learn: 2.0344015	test: 0.4022513	best: 0.4022513 (186)	total: 34.6s	remaining: 5m 35s
187:	learn: 2.0331104	test: 0.4018759	best: 0.4018759 (187)	total: 34.8s	remaining: 5m 35s
188:	learn: 2.0294737	test: 0.4012190	best: 0.4012190 (188)	total: 35s	remaining: 5m 34s
189:	learn: 2.0247119	test: 0.4003947	best: 0.4003947 (189)	total: 35.2s	remaining: 5m 35s
190:	learn: 2.0199867	test: 0.3998838	best: 0.3998838 (190)	total: 35.3s	remaining: 5m 34s
191:	learn: 2.0166824	test: 0.3991539	best: 0.3991539 (191)	total: 35.5s	remaining: 5m 34s
192:	learn: 2.0132543	test: 0.3984817	best: 0.3984817 (192)	total: 35.7s	remaining: 5m 34s
193:	learn: 2.0110060	test: 0.3975771	best: 0.3975771 (193)	total: 35.8s	remaining: 5m 33s
194:	learn: 2.0078306	test: 0.3965174	best: 0.3965174 (194)	total: 36s	remaining: 5m 33s
195:	learn: 2.0037295	test: 0.3968373	best: 0.3965174 (194)	total: 36.2s	remaining: 5m 33s
196

...

1629:	learn: 1.5733702	test: 0.3860522	best: 0.3664448 (484)	total: 6m 41s	remaining: 1m 31s
1630:	learn: 1.5732497	test: 0.3860322	best: 0.3664448 (484)	total: 6m 41s	remaining: 1m 30s
1631:	learn: 1.5732712	test: 0.3860211	best: 0.3664448 (484)	total: 6m 41s	remaining: 1m 30s
1632:	learn: 1.5730532	test: 0.3858774	best: 0.3664448 (484)	total: 6m 42s	remaining: 1m 30s
1633:	learn: 1.5728166	test: 0.3859425	best: 0.3664448 (484)	total: 6m 42s	remaining: 1m 30s
1634:	learn: 1.5729269	test: 0.3859758	best: 0.3664448 (484)	total: 6m 43s	remaining: 1m 29s
1635:	learn: 1.5730861	test: 0.3860275	best: 0.3664448 (484)	total: 6m 43s	remaining: 1m 29s
1636:	learn: 1.5730732	test: 0.3860803	best: 0.3664448 (484)	total: 6m 43s	remaining: 1m 29s
1637:	learn: 1.5730326	test: 0.3861120	best: 0.3664448 (484)	total: 6m 43s	remaining: 1m 29s
1638:	learn: 1.5729944	test: 0.3861499	best: 0.3664448 (484)	total: 6m 43s	remaining: 1m 28s
1639:	learn: 1.5730588	test: 0.3859623	best: 0.3664448 (484)	total: 6m

1718:	learn: 1.5701946	test: 0.3869421	best: 0.3664448 (484)	total: 7m 13s	remaining: 1m 10s
1719:	learn: 1.5699929	test: 0.3869559	best: 0.3664448 (484)	total: 7m 13s	remaining: 1m 10s
1720:	learn: 1.5697232	test: 0.3869330	best: 0.3664448 (484)	total: 7m 13s	remaining: 1m 10s
1721:	learn: 1.5698163	test: 0.3866984	best: 0.3664448 (484)	total: 7m 13s	remaining: 1m 10s
1722:	learn: 1.5698198	test: 0.3866875	best: 0.3664448 (484)	total: 7m 14s	remaining: 1m 9s
1723:	learn: 1.5698461	test: 0.3866801	best: 0.3664448 (484)	total: 7m 14s	remaining: 1m 9s
1724:	learn: 1.5696507	test: 0.3866932	best: 0.3664448 (484)	total: 7m 14s	remaining: 1m 9s
1725:	learn: 1.5696000	test: 0.3866726	best: 0.3664448 (484)	total: 7m 14s	remaining: 1m 8s
1726:	learn: 1.5696166	test: 0.3866625	best: 0.3664448 (484)	total: 7m 14s	remaining: 1m 8s
1727:	learn: 1.5697027	test: 0.3864613	best: 0.3664448 (484)	total: 7m 14s	remaining: 1m 8s
1728:	learn: 1.5695698	test: 0.3863889	best: 0.3664448 (484)	total: 7m 15s	r

1808:	learn: 1.5670854	test: 0.3863401	best: 0.3664448 (484)	total: 7m 50s	remaining: 49.7s
1809:	learn: 1.5670941	test: 0.3863230	best: 0.3664448 (484)	total: 7m 50s	remaining: 49.4s
1810:	learn: 1.5671284	test: 0.3862554	best: 0.3664448 (484)	total: 7m 50s	remaining: 49.1s
1811:	learn: 1.5670815	test: 0.3862849	best: 0.3664448 (484)	total: 7m 51s	remaining: 48.9s
1812:	learn: 1.5671222	test: 0.3863213	best: 0.3664448 (484)	total: 7m 51s	remaining: 48.6s
1813:	learn: 1.5665904	test: 0.3863388	best: 0.3664448 (484)	total: 7m 51s	remaining: 48.4s
1814:	learn: 1.5666353	test: 0.3863877	best: 0.3664448 (484)	total: 7m 52s	remaining: 48.1s
1815:	learn: 1.5666268	test: 0.3864070	best: 0.3664448 (484)	total: 7m 52s	remaining: 47.9s
1816:	learn: 1.5666613	test: 0.3863791	best: 0.3664448 (484)	total: 7m 52s	remaining: 47.6s
1817:	learn: 1.5666139	test: 0.3863594	best: 0.3664448 (484)	total: 7m 53s	remaining: 47.4s
1818:	learn: 1.5665742	test: 0.3863842	best: 0.3664448 (484)	total: 7m 53s	remai

1898:	learn: 1.5640186	test: 0.3861239	best: 0.3664448 (484)	total: 8m 24s	remaining: 26.8s
1899:	learn: 1.5641323	test: 0.3861132	best: 0.3664448 (484)	total: 8m 24s	remaining: 26.6s
1900:	learn: 1.5640321	test: 0.3860919	best: 0.3664448 (484)	total: 8m 25s	remaining: 26.3s
1901:	learn: 1.5641240	test: 0.3861037	best: 0.3664448 (484)	total: 8m 25s	remaining: 26.1s
1902:	learn: 1.5641119	test: 0.3861293	best: 0.3664448 (484)	total: 8m 26s	remaining: 25.8s
1903:	learn: 1.5640586	test: 0.3863602	best: 0.3664448 (484)	total: 8m 26s	remaining: 25.6s
1904:	learn: 1.5640344	test: 0.3863464	best: 0.3664448 (484)	total: 8m 27s	remaining: 25.3s
1905:	learn: 1.5639116	test: 0.3864287	best: 0.3664448 (484)	total: 8m 27s	remaining: 25s
1906:	learn: 1.5638837	test: 0.3864815	best: 0.3664448 (484)	total: 8m 28s	remaining: 24.8s
1907:	learn: 1.5637042	test: 0.3864867	best: 0.3664448 (484)	total: 8m 28s	remaining: 24.5s
1908:	learn: 1.5637021	test: 0.3864956	best: 0.3664448 (484)	total: 8m 29s	remaini

1988:	learn: 1.5609246	test: 0.3863343	best: 0.3664448 (484)	total: 9m 2s	remaining: 3s
1989:	learn: 1.5609789	test: 0.3863577	best: 0.3664448 (484)	total: 9m 3s	remaining: 2.73s
1990:	learn: 1.5609875	test: 0.3863413	best: 0.3664448 (484)	total: 9m 4s	remaining: 2.46s
1991:	learn: 1.5608752	test: 0.3863444	best: 0.3664448 (484)	total: 9m 4s	remaining: 2.19s
1992:	learn: 1.5610367	test: 0.3864359	best: 0.3664448 (484)	total: 9m 4s	remaining: 1.91s
1993:	learn: 1.5608890	test: 0.3863616	best: 0.3664448 (484)	total: 9m 5s	remaining: 1.64s
1994:	learn: 1.5609782	test: 0.3863586	best: 0.3664448 (484)	total: 9m 6s	remaining: 1.37s
1995:	learn: 1.5609082	test: 0.3863763	best: 0.3664448 (484)	total: 9m 6s	remaining: 1.09s
1996:	learn: 1.5609345	test: 0.3863477	best: 0.3664448 (484)	total: 9m 7s	remaining: 822ms
1997:	learn: 1.5609698	test: 0.3863489	best: 0.3664448 (484)	total: 9m 7s	remaining: 548ms
1998:	learn: 1.5610086	test: 0.3863285	best: 0.3664448 (484)	total: 9m 8s	remaining: 274ms
19

In [103]:
train_p_2 = model2.predict(train_pool_2)
train['delta_2'] = ((train[target_col] - train_p_2)/ train_p_2)
corr_koef_city_2 = dict(train.query("price_type == 1").groupby(['city']).delta_2.median())
corr_koef_region_2 = dict(train.query("price_type == 1").groupby(['region']).delta_2.median())
corr_koef_2 = train.query("price_type == 1").groupby(['city']).delta_2.median().median()

In [108]:
valid_p_2 = model2.predict(valid_pool_2)
valid['corr_koef_2'] = (valid[['city', 'region']]
                          .apply(lambda x: corr_koef_city_2.get(x[0], corr_koef_region_2.get(x[1], corr_koef_2)),
                                 axis=1)
                      + 1)
valid_p_2 = (valid_p_2 * valid['corr_koef_2'])* .9

In [109]:
deviation_metric(valid[target_col].values, valid_p_2.values), deviation_metric(train[target_col].values, train_p_2)

(1.6731232013791448, 3.7657726150604596)

In [110]:
test_p = model2.predict(test[cols2])
test['corr_koef'] = (test[['city', 'region']]
                          .apply(lambda x: corr_koef_city_2.get(x[0], corr_koef_region_2.get(x[1], corr_koef_2)),
                                 axis=1)
                      + 1)
test_p = (test_p * test['corr_koef']) * .9

In [112]:
test[target_col] = test_p
test[['id', target_col]].to_csv("./output/try_0_0.csv", sep=',', header=True, index=False)

In [30]:
tmp = train.merge(fillna_val.reset_index(drop=True), on='region', suffixes=('', '_fill'))